In [ ]:
We use Ubuntu on Amazon EC2 as our web service. Our public DNS name is ec2-35-167-59-55.us-west-2.compute.amazonaws.com. 
The public elastic IP address is 35.167.59.55.

In [ ]:
1.	We installed sqlite3 on Ubuntu.
2.	In this homework, we write down a python script to import CSV file and then create database and tables 
    in this database and then import data from CSV file to databases. 
    (eg: We use locations.py to import data from properties_2016_test.csv to locations_v2.db)
    
    locations.py:
        import csv
        import sqlite3

        conn = sqlite3.connect('locations_v2.db')
        cur = conn.cursor()
        cur.execute("""DROP TABLE IF EXISTS locations""")
        cur.execute("""CREATE TABLE locations
                    (parcelid text, latitude text, longitude text)""")

        with open('properties_2016_location.csv', 'r') as f:
            reader = csv.reader(f.readlines()[1:])  # exclude header line
            cur.executemany("""INSERT INTO locations VALUES (?,?,?)""",
                            (row for row in reader))
        conn.commit()
        conn.close()
3. We need to connect to the database and use query to return data from database:
    DBConnection.py connect to locations.db this database:
        import sqlite3
        import os.path
        from flask import Flask, Response, request, g

        app = Flask(__name__)

        DATABASE = '/home/ubuntu/info7390-A2/locations.db'
        app.config.from_object(__name__)

        def connect_to_database():
            return sqlite3.connect(app.config['DATABASE'])

        def get_db():
            db = getattr(g, 'db', None)
            if db is None:
                db = g.db = connect_to_database()
            return db

        @app.teardown_appcontext
        def close_connection(exception):
            db = getattr(g, 'db', None)
            if db is not None:
                db.close()

        def execute_query(query, args=()):
            cur = get_db().execute(query, args)
            rows = cur.fetchall()
            cur.close()
            return rows


        if __name__ == '__main__':
          app.run()
    
    
    in GeospatialSearch3.py we use query to retrieve data from database:
        def getLocations(origin, number, list):
            millis1 = int(round(time.time() * 1000))
            rows = DBConnection.execute_query("""SELECT * FROM locations""")
            for row in rows:
                longitude =float( int(row[2]))/1000000
                latitude = float(int(row[1]))/1000000
                point = [longitude, latitude]
                distance = getDistance(origin, point)
                if len(list) < number:
                    newlocation = Location(row[0], longitude, latitude, distance)
                    list.append(newlocation)
                    list.sort()
                elif distance < list[number - 1].distance:
                    del list[number - 1]
                    newlocation = Location(row[0], longitude, latitude, distance)
                    list.append(newlocation)
                    list.sort()
                else:
                    continue
            millis2 = int(round(time.time() * 1000))
            time1 = float(millis2 - millis1) / 1000
            generateResponse(list, time1)